In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/result/submission.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
!pip install -q sentence-transformers

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
import pandas as pd
data_path = '/kaggle/input/nlp-getting-started/train.csv'
df = pd.read_csv(data_path)
df_test = df.sample(int(len(df)*0.2))
df_test

,id,keyword,location,text,target
50,73,ablaze,"Sheffield Township, Ohio",Deputies: Man shot before Brighton home set ab...,1
7123,10204,violent%20storm,Barbados,Dramatic Video Shows Plane Landing During Viol...,1
2573,3690,destroy,Norway,Politics = Preschool Attitude: Russia orders t...,1
2402,3459,derailed,"Washington, DC",Happy no one was hurt when #wmata train derail...,1
212,299,annihilated,Boksburg,@marksmaponyane Hey!Sundowns were annihilated ...,0
...,...,...,...,...,...
5074,7232,natural%20disaster,NaN,Hello natural hazards/disaster recovery &amp; ...,0
3885,5524,flattened,UK,Today (August 6th) is the 70th anniversary of ...,1
704,1017,blazing,Konoha,@__srajapakse__ Why thank you there missy ?? t...,0
5387,7687,panic,Toronto,tomorrow's going to be a year since I went to ...,1


In [5]:
df_train = df.drop(df_test.index)
df_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-en-v1.5')
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

2024-08-14 12:27:18.463477: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 12:27:18.463594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 12:27:18.608108: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        text = row['text']
        label = row['target']
        
        # Tokenize the text
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Convert to tensors
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label, dtype=torch.long)
        }


In [8]:
train_dataset = CustomDataset(df_train, tokenizer)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
val_dataset = CustomDataset(df_test, tokenizer)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=4)

In [9]:
from sentence_transformers import SentenceTransformer, models, losses, InputExample
from torch import nn
from torch.utils.data import DataLoader

class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassificationHead, self).__init__()
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, features):
        x = features['sentence_embedding']
        x = self.linear(x)
        return x

In [10]:
class SentenceTransformerWithHead(nn.Module):
    def __init__(self, sentence_transformer, classification_head):
        super(SentenceTransformerWithHead, self).__init__()
        self.sentence_transformer = sentence_transformer
        self.classification_head = classification_head
    
    def forward(self, input_ids, attention_mask):
        features = self.sentence_transformer({'input_ids': input_ids, 'attention_mask': attention_mask})
        logits = self.classification_head(features)
        return logits

In [11]:
embedding_dim = 1024
num_classes = 2
classification_head = ClassificationHead(embedding_dim,num_classes)

In [ ]:
import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

model_with_head = SentenceTransformerWithHead(model,classification_head).to(device)
optimizer = AdamW(model_with_head.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()

# Fine-tuning loop
for epoch in range(3):  # Number of epochs
    model_with_head.train()
    total_loss = 0
    total_corrected = 0
    num_trained = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model_with_head(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_corrected += sum(outputs.argmax(dim=-1) == labels)
        total_loss += loss.item() * labels.shape[0]
        num_trained += labels.shape[0]
    print(f'Epoch {epoch+1} completed, train loss: {total_loss/num_trained}, train accuracy: {total_corrected/num_trained}')
    print('\nDo Validation\n')
    model_with_head.eval()
    val_loss = 0
    val_corrected = 0
    num_val = 0
    for val_batch in val_dataloader:
        val_input_ids = val_batch['input_ids'].to(device)
        val_attention_mask = val_batch['attention_mask'].to(device)
        val_labels = val_batch['label'].to(device)
        val_outputs = model_with_head(input_ids=val_input_ids, attention_mask=val_attention_mask)
        val_loss = loss_fn(val_outputs, val_labels)
        val_corrected += sum(val_outputs.argmax(dim=-1) == val_labels)
        val_loss += val_loss.item() * val_labels.shape[0]
        num_val += val_labels.shape[0]
    print(f'Validation completed, val loss: {val_loss/num_val}, val accuracy: {val_corrected/num_val}')
    print('===========================================================================================================================')
# Save the fine-tuned model
torch.save(model_with_head.state_dict(), 'finetuned_model.pth')

Epoch 1 completed, train loss: 0.537248522933726, train accuracy: 0.7947791814804077

Do Validation

